In [24]:
import random 
from keras.models import load_model
from music21 import converter, instrument, note, chord, stream
import numpy as np
import pickle
from tqdm import tqdm

In [26]:
#WITHOUT INSTRUMNET
def read_ts_midi(file_path, timestep): 
    notes = [] 
    notes_to_parse = None 


    midi= converter.parse(file_path)

    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
#         if 'Piano' in str(part): 

        notes_to_parse = part.recurse() 

        #finding whether a particular element is note or a chord
        for element in notes_to_parse:

            #note
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))

            #chord
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    notes = np.array(notes)
#     np.random.shuffle(notes)
    return notes[:timestep]

        
def get_pickles(pick_name): 
    x_train, x_test, y_train, y_test, notes_dict = pickle.load(open(f'../Pickles/{pick_name}', 'rb'))
    return x_train, x_test, y_train, y_test, notes_dict



def convert_to_midi(prediction_output, filename):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                output_notes.append(instrument.SnareDrum())
                cn=current_note
                new_note = note.Note(cn)
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        else:
            output_notes.append(instrument.SnareDrum())
            new_note = note.Note(pattern)
            new_note.offset = offset
#             new_note.storedInstrument = instrument.Violin()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += .5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)


def generate_music(model, first_sequence, timestep = 50, total_runs = 50):
    song = [i for i in first_sequence]
    total_iter = 0
    current_idx = 0
    pbar = tqdm(desc= 'Creating Music With Neural Network', total = total_runs)
    while total_iter < total_runs:
        current_seq = song[current_idx: timestep+current_idx]
        current_seq = np.array(current_seq).reshape(1, timestep, 1)
        next_note = np.argmax(model.predict(current_seq)[0])
        song.append(next_note)
        total_iter += 1 
        current_idx += 1
        pbar.update(1)
    
    return song

# #Random generated
# test_type = 'Tester'
# x_train, x_test, y_train, y_test, notes_dict = get_pickles(f'{test_type}.p')
# reverse_notes = {n:idx for idx, n in notes_dict.items()}
# first_seq = np.random.randint(0, len(reverse_notes), size = 50)
# model = load_model(f'ModelWeights/LSTM_{test_type}.h5')

# new_song = generate_music(model, first_seq, total_runs = 50)
# new_song_notes = [notes_dict[i] for i in new_song]
    
# convert_to_midi(new_song_notes, filename = f'Tests/{test_type}.mid')
    
    
# first sequence from a song 
test_type = 'Tester'
x_train, x_test, y_train, y_test, notes_dict = get_pickles(f'{test_type}.p')
reverse_notes = {n:idx for idx, n in notes_dict.items()}
first_seq = read_ts_midi('../Classical/004311b_.mid', timestep = 50)
print(first_seq)
first_seq = [reverse_notes[i] for i in first_seq]
print(first_seq)
model = load_model(f'ModelWeights/LSTM_{test_type}.h5')

new_song = generate_music(model, first_seq, total_runs = 300)
new_song_notes = [notes_dict[i] for i in new_song]
    
convert_to_midi(new_song_notes, filename = f'Tests/{test_type}.mid')

['G4' 'D4' 'B3' 'G3' 'G4' 'E4' 'B3' 'E3' 'A4' 'F#4' 'D4' 'D3' 'B4' 'G4'
 'D4' 'G3' 'C5' 'C4' 'A3' 'D5' 'B3' 'B3' 'D5' 'G4' 'B3' 'G3' 'D5' 'E4'
 'A3' 'A3' 'C#5' 'E4' 'A3' 'A2' 'D5' 'F#4' 'A3' 'D3' 'B4' 'G4' 'G3' 'G3'
 'C5' 'E4' 'G3' 'C3' 'B4' 'D4' 'G3' 'G2']
[901, 166, 1322, 346, 901, 1200, 1322, 389, 1656, 350, 166, 999, 344, 901, 166, 346, 208, 1669, 316, 857, 1322, 1322, 857, 901, 1322, 346, 857, 1200, 316, 316, 173, 1200, 316, 780, 857, 350, 316, 999, 344, 901, 346, 346, 208, 1200, 346, 1606, 344, 166, 346, 380]


Creating Music With Neural Network: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:22<00:00, 44.64it/s]


In [ ]:
notes_dict

In [ ]:
#WITH INSTRUMENT
def read_ts_midi(file_path, timestep): 
    notes = [] 
    notes_to_parse = None 
    instruments = []

    midi= converter.parse(file_path)
    
    s2 = instrument.partitionByInstrument(midi)
    for part in s2.parts:
        str_part = str(part)
        if str_part == '<music21.stream.Part>': 
            continue 
        else:
            cutoff_len = len('<music21.stream.Part ') 
            instru = str_part[cutoff_len:]
            instru = instru[:instru.find('>')]
            notes_to_parse = part.recurse() 
            for element in notes_to_parse: 
                if isinstance(element, note.Note): 
                    notes.append(str(element.pitch))
                    instruments.append(instru)
                elif isinstance(element, chord.Chord): 
                    notes.append('.'.join(str(n) for n in element.normalOrder))      
                    instruments.append(instru)
                    
    return np.array(list(zip(instruments, notes)))
        
def get_pickles(pick_name): 
    x_train, x_test, y_train, y_test, notes_dict = pickle.load(open(f'../Pickles/{pick_name}', 'rb'))
    
    return x_train, x_test, y_train, y_test, notes_dict






def generate_music(model, first_sequence, timestep = 10, total_runs = 25):
    song = [i for i in first_sequence]
    total_iter = 0
    current_idx = 0
    while total_iter < total_runs: 
        current_seq = song[current_idx: timestep+current_idx]
        current_seq = np.array(current_seq).reshape(1, timestep, 1)
        next_note = np.argmax(model.predict(current_seq)[0])
        song.append(next_note)
        total_iter += 1 
        current_idx += 1
    
    return song
            



x_train, x_test, y_train, y_test, notes_dict = get_pickles('50cent.p')
reverse_notes = {n:idx for idx, n in notes_dict.items()}
first_seq = read_ts_midi('../50Cent/50_cent-piggy_bank.mid', timestep = 10)
first_seq = [' '.join(i) for i in first_seq]
first_seq = [reverse_notes[i] for i in first_seq]
model = load_model('ModelWeights/LSTM.h5')

new_song = generate_music(model, first_seq)
new_song_notes = [notes_dict[i] for i in new_song]


In [ ]:
new_song_notes

In [ ]:
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for ints_note in prediction_output:
        ints_note_split = ints_note.split(' ')
        pattern = ints_note_split[-1]
        instru = ' '.join(ints_note_split[:-1])
        print(instru)
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                output_notes.append(instrument.Instrument(instrumentName = instru))
                cn=int(current_note)
                new_note = note.Note(cn)
#                 new_note.storedInstrument = instrument.Violin()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            output_notes.append(instrument.Instrument(instrumentName = instru))

            new_note = note.Note(pattern)
            new_note.offset = offset
#             new_note.storedInstrument = instrument.Violin()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += .5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')
    
convert_to_midi(new_song_notes)

In [ ]:

print(new_song_notes)
print(new_song)

In [ ]:
idx = np.random.randint(0, len(x_test)-1)
random_music = x_test[idx]
predictions = [] 
for i in range(10): 
    random_music = random_music.reshape(1,timestep,1)
    prob = lstm_model.predict(random_music)[0]
    y_pred = np.argmax(prob, axis = 0)
    
    predictions.append(y_pred)
    
    random_music = np.insert(random_music[0], len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

unique_x = list(set(x.ravel()))

x_int_dict = dict((i,n) for n,i in enumerate(unique_x))

predicted_notes = [x_int_dict[i] for i in predictions]
predicted_notes